# Loading And Fetching Data From Reddit PRAW

In [ ]:
SECRET_KEY = 'LyyFU_r17F6s1i0ajYI2dxoSi2dOtw'
CLIENT_ID = 'SMi4R1E-3TeMoXqqwPNwUg'
USER_NAME = 'Weary-Tooth7440'

In [2]:
%pip install praw

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [8]:
import praw

# Initialize Reddit instance
reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=SECRET_KEY,
                     user_agent=USER_NAME)

In [9]:
# Subreddit to scrape
subreddit = reddit.subreddit('stocks')

# Display the name of the Subreddit
print("Display Name:", subreddit.display_name)
 
# Display the title of the Subreddit
print("Title:", subreddit.title)
 
# Display the description of the Subreddit
print("Description:", subreddit.description)

Display Name: stocks
Title: Stocks - Investing and trading for all
Description: Almost any post related to stocks is welcome; please read the rules below:

[**If you're new here**](https://www.reddit.com/r/stocks/comments/4x1419/if_youre_new_here_read_this_post_first/)

##Resources

* [Wiki for new investors](https://www.reddit.com/r/stocks/wiki/index)

* [Pattern day trading](https://www.reddit.com/r/stocks/wiki/pdtrules)

* [Earnings calendar](https://finance.yahoo.com/calendar/earnings/)

##Karma requirements

[Click here to find how many days old your account needs to be and how much karma you need](https://www.reddit.com/r/stocks/wiki/karma) before you can comment or post to r/Stocks.

##Rules [(in depth rules wiki here)](https://www.reddit.com/r/stocks/wiki/rules)

1. Disclose any related open positions when discussing a particular stock or financial instrument.

2. Spam, ads, solicitations (including referral links), and self-promotion posts or comments will be removed and you m

In [14]:
# assume you have a Subreddit instance bound to variable `subreddit`
for submission in subreddit.hot(limit=10):
    print(submission.title)
    # Output: the submission's title
    print(submission.score)
    # Output: the submission's score
    print(submission.id)
    # Output: the submission's ID
    print(submission.url)
    # Output: the URL the submission points to or the submission's URL if it's a self post

Rate My Portfolio - r/Stocks Quarterly Thread June 2024
55
1d5itn0
https://www.reddit.com/r/stocks/comments/1d5itn0/rate_my_portfolio_rstocks_quarterly_thread_june/
r/Stocks Daily Discussion Wednesday - Jul 17, 2024
9
1e5dx4m
https://www.reddit.com/r/stocks/comments/1e5dx4m/rstocks_daily_discussion_wednesday_jul_17_2024/
Global chip stocks from Nvidia to ASML fall on geopolitics, Trump comments
551
1e5eubd
https://www.reddit.com/r/stocks/comments/1e5eubd/global_chip_stocks_from_nvidia_to_asml_fall_on/
These are the stocks on my watchlist (7/17)  
16
1e5hleu
https://www.reddit.com/r/stocks/comments/1e5hleu/these_are_the_stocks_on_my_watchlist_717/
ASML Holding GAAP EPS of €4.01, revenue of €6.2B; initiates Q3 and reaffirms FY24 outlook
20
1e5fovo
https://www.reddit.com/r/stocks/comments/1e5fovo/asml_holding_gaap_eps_of_401_revenue_of_62b/
Why does semiconductor companies such as TSMC continue to break all-time high?
35
1e5bdvb
https://www.reddit.com/r/stocks/comments/1e5bdvb/why_does_se

In [17]:
import pprint

# assume you have a praw.Reddit instance bound to variable `reddit`
submission = reddit.submission("39zje0")
print(submission.title)  # to make it non-lazy
pprint.pprint(vars(submission))

reddit will soon only be available over HTTPS
{'_additional_fetch_params': {},
 '_comments': <praw.models.comment_forest.CommentForest object at 0x00000197C3E6EC50>,
 '_comments_by_id': {'t1_cs7vwlm': Comment(id='cs7vwlm'),
                     't1_cs7xcx2': Comment(id='cs7xcx2'),
                     't1_cs7ykx6': Comment(id='cs7ykx6'),
                     't1_cs81mem': Comment(id='cs81mem'),
                     't1_cs81xp8': Comment(id='cs81xp8'),
                     't1_cs82epc': Comment(id='cs82epc'),
                     't1_cs82kes': Comment(id='cs82kes'),
                     't1_cs83dd8': Comment(id='cs83dd8'),
                     't1_cs83ua8': Comment(id='cs83ua8'),
                     't1_cs83xhc': Comment(id='cs83xhc'),
                     't1_cs846jk': Comment(id='cs846jk'),
                     't1_cs847yp': Comment(id='cs847yp'),
                     't1_cs848n2': Comment(id='cs848n2'),
                     't1_cs84apf': Comment(id='cs84apf'),
                     '

In [22]:
# Importing necessary libraries
import pandas as pd
import time

# Define lists to store data
data = []

# Scraping posts & Comments
for post in subreddit.top(limit= 10): 
    #data.append({
    #    'Type': 'Post',
    #    'Post_id': post.id,
    #    'Title': post.title,
    #    'Author': post.author.name if post.author else 'Unknown',
    #    'Timestamp': post.created_utc,
    #    'Text': post.selftext,
    #    'Score': post.score,
    #    'Total_comments': post.num_comments,
    #    'Post_URL': post.url
    #})

# Check if the post has comments
    if post.num_comments > 0:
        # Scraping comments for each post
        post.comments.replace_more(limit= 5)
        for comment in post.comments.list():
            data.append({
                'id': post.id + '_' +  comment.id ,
                'Title': post.title,
                'Author': comment.author.name if comment.author else 'Unknown',
                'Timestamp': pd.to_datetime(comment.created_utc, unit='s'),
                'Text': comment.body,
                'Score': comment.score,
                'Post_url':post.url,

            })
            
    time.sleep(2)  # Pause for 2 second between requests


# Create pandas DataFrame for posts and comments
df = pd.DataFrame(data)

In [23]:
df

,id,Title,Author,Timestamp,Text,Score,Post_url
0,la34bh_glls7c9,"It's fucking awful seeing the ""Silver"" misinfo...",provoko,2021-02-01 14:58:37,"~~Just a small correction~~, FYI you can short...",1,https://www.reddit.com/r/stocks/comments/la34b...
1,la34bh_gllicpw,"It's fucking awful seeing the ""Silver"" misinfo...",SmithRune735,2021-02-01 13:45:17,So they went from saying redditors are bad for...,3933,https://www.reddit.com/r/stocks/comments/la34b...
2,la34bh_gllhth2,"It's fucking awful seeing the ""Silver"" misinfo...",Unknown,2021-02-01 13:40:32,Many of the misinformation lies on the fact th...,1471,https://www.reddit.com/r/stocks/comments/la34b...
3,la34bh_glliht7,"It's fucking awful seeing the ""Silver"" misinfo...",stop-spending-money,2021-02-01 13:46:31,Owning a physical silver bar would be pretty c...,522,https://www.reddit.com/r/stocks/comments/la34b...
4,la34bh_glli4fd,"It's fucking awful seeing the ""Silver"" misinfo...",elguapo2385,2021-02-01 13:43:15,Ape no like silver. \nApe like movies and game...,1183,https://www.reddit.com/r/stocks/comments/la34b...
...,...,...,...,...,...,...,...
8332,lazcak_glt5p4x,What $GME has taught me in 36 hours of day tra...,Axion132,2021-02-03 01:38:04,It's hard to say the play is over when there a...,5,https://www.reddit.com/r/stocks/comments/lazca...
8333,lazcak_glso5k6,What $GME has taught me in 36 hours of day tra...,Theta_God,2021-02-02 23:18:07,Agreed. The only thing left is a major catalys...,4,https://www.reddit.com/r/stocks/comments/lazca...
8334,lazcak_gls6qsq,What $GME has taught me in 36 hours of day tra...,Unknown,2021-02-02 21:17:37,[deleted],10,https://www.reddit.com/r/stocks/comments/lazca...
8335,lazcak_glsi3hb,What $GME has taught me in 36 hours of day tra...,Ikea_Man,2021-02-02 22:33:18,">I don't gamble and am very risk averse, but I...",8,https://www.reddit.com/r/stocks/comments/lazca...


# Preprocessing Comment